# Add-One Smoothing

Here, I take two English corpora - the King James Version of the Bible and the Universal Declaration of Human Rights - and apply add-one smoothing to generate n-gram models for each corpus. For simplicity, all words will be in lowercase and the models will be case-insensitive.

Add-one smoothing is a method of computing the probability of a word in a n-gram model in such a way that the sequences that never appear in the corpus do not get zero probability.

$$
\begin{align*}
p &= \frac{c + 1}{n + v} \\
c &= \textrm{count of the n-gram} \\
n &= \textrm{count of the history (the n-gram excluding the last word)} \\
v &= \textrm{size of the vocabulary}
\end{align*}
$$

For each model, I compute its cross-entropy and perplexity and develop a simple sentence finisher. After that, I analyze the effectiveness of each corpus at training n-gram models.

In [1]:
%cd ..

/home/mtj0712/Documents/playground


/home/mtj0712/.local/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import re
import math

import pygtrie
import numpy as np
from numpy import random

from reader import *

`punc_pattern` will help us separate the punctuations from actual words.

In [3]:
punc_pattern = re.compile('[’!\"#$%&\'()*+,\\-./:;<=>?@[\\\\\\]^_`{|}~]')
end_mark_set = {'!', '.', '?'}

## King James Version

First, I build n-gram models with the King James Version of the Bible. This will give us a language model for Early Modern English.

In [4]:
kjv = KJVReader()
kjv_list = []

Before building the models, I parse the text into a list of words and punctuations. This will be convenient for building the models.

In [5]:
while not kjv.is_eof():
    units = kjv.read_sentence().lower().split()
    for u in units:
        while u:
            match = punc_pattern.search(u)
            if match:
                i = match.start()
                if i == 0:
                    punc = u[0]
                    u = u[1:]
                else:
                    first_word = u[:i]
                    punc = u[i]
                    u = u[i+1:]
                    kjv_list.append(first_word)
                kjv_list.append(punc)
            else:
                kjv_list.append(u)
                break

First, I build the unigram model without add-one smoothing.

In [6]:
# Unigram Model

kjv_trie = pygtrie.StringTrie()
kjv_v = 0 # size of the vocabulary
kjv_wordlist = []

for w in kjv_list:
    try:
        kjv_trie[w] += 1
    except KeyError:
        kjv_trie[w] = 1
        kjv_wordlist.append(w)
        kjv_v += 1

print('Size of the vocabulary:', kjv_v)
print('Count of all words:', len(kjv_list))

kjv_unigram_H = 0 # cross entropy

for w in kjv_list:
    p = kjv_trie[w] / len(kjv_list)
    kjv_unigram_H += math.log2(p)
kjv_unigram_H /= -len(kjv_list)

print('Cross entropy:', kjv_unigram_H)
print('Perplexity:', 2 ** kjv_unigram_H)
print()

kjv_unigram_list = sorted(kjv_trie.items(), key=lambda t : t[1], reverse=True)
for pair in kjv_unigram_list:
    p = pair[1] / len(kjv_list)
    print('Word:', pair[0], '| Probability:', p)

Size of the vocabulary: 12557
Count of all words: 917082
Cross entropy: 8.298804717355264
Perplexity: 314.91195501035224

Word: , | Probability: 0.07696040266846367
Word: the | Probability: 0.06970914269389215
Word: and | Probability: 0.05637009558578186
Word: of | Probability: 0.03774798763905518
Word: . | Probability: 0.028567783469744253
Word: to | Probability: 0.014788208687990823
Word: that | Probability: 0.014079438916040223
Word: : | Probability: 0.01384936134391472
Word: in | Probability: 0.013812287232766536
Word: he | Probability: 0.01136103423685123
Word: ; | Probability: 0.011055718027395587
Word: shall | Probability: 0.010727503102230772
Word: unto | Probability: 0.009810464058830072
Word: for | Probability: 0.009782113267952048
Word: i | Probability: 0.00965453470900094
Word: his | Probability: 0.00923908658113451
Word: a | Probability: 0.008916323731138546
Word: lord | Probability: 0.008684065328945504
Word: they | Probability: 0.008042901289088652
Word: be | Probability

The cross entropy and perplexity of the model is extremely high. This is expected, since a unigram model is far from sufficient in representing an actual language. As expected, the most probable words are some common punctuations and grammatical words, such as articles, prepositions, and pronouns.

Next, I build 2~5-gram models. Again, I do not apply add-one smoothing. This time, I do not print out the probabilities for each n-gram, since it would be too lengthy.

In [7]:
# 2~5-gram Model

for n in range(2, 6):
    print('========== ' + ('bi' if n == 2 else ('tri' if n == 3 else f'{n}-')) + 'gram ==========')
    print()
    
    ngram = [''] * n
    for w in kjv_list:
        ngram[-1] = w
        try:
            kjv_trie['/'.join(ngram)] += 1
        except KeyError:
            kjv_trie['/'.join(ngram)] = 1
    
        if w in end_mark_set:
            ngram[:-1] = [''] * (n - 1)
        else:
            ngram[:-1] = ngram[1:]
    
    kjv_ngram_H = 0 # cross entropy
    
    ngram = [''] * n
    for w in kjv_list:
        ngram[-1] = w
        if ngram[-2] == '':
            history_count = kjv_trie['.'] + kjv_trie['?'] + kjv_trie['!']
        else:
            history_count = kjv_trie['/'.join(ngram[:-1])]
        p = kjv_trie['/'.join(ngram)] / history_count
        kjv_ngram_H += math.log2(p)
        
        if w in end_mark_set:
            ngram[:-1] = [''] * (n - 1)
        else:
            ngram[:-1] = ngram[1:]
    kjv_ngram_H /= -len(kjv_list)
    
    print('Cross entropy:', kjv_ngram_H)
    print('Perplexity:', 2 ** kjv_ngram_H)
    print()

========== bigram ==========

Cross entropy: 5.519835563333419
Perplexity: 45.88133813806327

========== trigram ==========

Cross entropy: 3.4234831475210146
Perplexity: 10.729293289206742

========== 4-gram ==========

Cross entropy: 1.7690779742168263
Perplexity: 3.408360589060311

========== 5-gram ==========

Cross entropy: 0.9672991316460356
Perplexity: 1.9551768815847856



As expected, as the order of the n-gram model increases, the perplexity of the language model decreases. Now, I try the same while applying add-one smoothing.

In [8]:
print('========== unigram ==========')
print()

kjv_unigram_H = 0 # cross entropy

for w in kjv_list:
    p = (kjv_trie[w] + 1) / (len(kjv_list) + kjv_v)
    kjv_unigram_H += math.log2(p)
kjv_unigram_H /= -len(kjv_list)

print('Cross entropy:', kjv_unigram_H)
print('Perplexity:', 2 ** kjv_unigram_H)
print()

for n in range(2, 6):
    print('========== ' + ('bi' if n == 2 else ('tri' if n == 3 else f'{n}-')) + 'gram ==========')
    print()
    
    kjv_ngram_H = 0 # cross entropy
    
    ngram = [''] * n
    for w in kjv_list:
        ngram[-1] = w
        if ngram[-2] == '':
            history_count = kjv_trie['.'] + kjv_trie['?'] + kjv_trie['!']
        else:
            history_count = kjv_trie['/'.join(ngram[:-1])]
        p = (kjv_trie['/'.join(ngram)] + 1) / (history_count + kjv_v)
        kjv_ngram_H += math.log2(p)
        
        if w in end_mark_set:
            ngram[:-1] = [''] * (n - 1)
        else:
            ngram[:-1] = ngram[1:]
    kjv_ngram_H /= -len(kjv_list)
    
    print('Cross entropy:', kjv_ngram_H)
    print('Perplexity:', 2 ** kjv_ngram_H)
    print()

========== unigram ==========

Cross entropy: 8.301704011836414
Perplexity: 315.54545031215395

========== bigram ==========

Cross entropy: 8.326024279798116
Perplexity: 320.9098539312886

========== trigram ==========

Cross entropy: 10.415587108696398
Perplexity: 1365.8535701063633

========== 4-gram ==========

Cross entropy: 11.458426499090292
Perplexity: 2814.0387879465566

========== 5-gram ==========

Cross entropy: 11.853803915153735
Perplexity: 3701.268074297006



When add-one smoothing is applied, contrary to our expectation, the perplexity of the language model increases as the order of the n-gram model increases. This might be because at higher orders of the n-gram model, the count of the history before each word ($n$) is smaller, and this causes the size of the vocabulary ($v$) to comprise a bigger portion in the denominator of the probability fraction.

It seems that add-one smoothing is only useful for dealing with new texts, and not for computing the cross-entropy or the perplexity of a language model with the very corpus it was trained on.

Below, I implement a sentence finisher with 2~5-gram models developed above. New word is generated by randomly choosing from the corpus vocabulary with the probability calculated with add-one smoothing. Whenever the history of an n-gram cannot be found in the corpus, I reduce the order of the n-gram by $1$. After that, I try to increase the order back to `n`.

With each model, I will try to finish the sequence "i will".

In [15]:
for n in range(2, 6):
    print('========== ' + ('bi' if n == 2 else ('tri' if n == 3 else f'{n}-')) + 'gram ==========')
    print()
    
    ngram = [''] * n
    ngram[-2:] = ['i', 'will']
    print(' '.join(ngram[-2:]), end='')

    sentence_len = 2

    start = 0 # for finding the starting point of history
    
    while ngram[-1] not in end_mark_set:
        if 200 < sentence_len:
            print(' <<< The sentence being generated has exceeded 200 tokens. Sentence finishing failed. >>>')
            break
        
        ngram[:-1] = ngram[1:]
        
        if ngram[-2] == '':
            history = '/' * (n - 2)
            history_count = kjv_trie['.'] + kjv_trie['?'] + kjv_trie['!']
        else:
            if 0 < start:
                start -= 1
            
            for i in range(start, n - 1):
                history = '/'.join(ngram[i:-1])
                try:
                    history_count = kjv_trie[history]
                    start = i
                    break
                except KeyError:
                    pass

        p = np.zeros(kjv_v)
        
        for i in range(kjv_v):
            try:
                count = kjv_trie[history + '/' + kjv_wordlist[i]]
            except KeyError:
                count = 0
            p[i] = (count + 1) / (history_count + kjv_v)
        
        ngram[-1] = kjv_wordlist[random.choice(kjv_v, p=p)]

        if punc_pattern.match(ngram[-1]) or ngram[-2] == '-' or ngram[-2:] == ['’', 's']:
            output = ngram[-1]
        else:
            output = ' ' + ngram[-1]
        print(output, end='')

        sentence_len += 1
    
    print()
    print()

========== bigram ==========

i will bashan waterflood cheerfully fugitive cloak elidad mutual underneath unknown bits acquit first hen zimri winebibber anguish linus retain gazing rabbi ephrathites conclude scholar lachish horims kerchiefs elnathan ammihud yours brands seled remmon traditions choked orchards rehoboth attained anakims compassion blew industrious criest thereat ha lingereth followeth shutteth entangled writer quickeneth scorched excel realm coucheth nun edge adders iniquity elisha shore dale lost jehonathan trogyllium semachiah baskets produce pennyworth canker tahan abound baalshalisha shimron perazim miletus halted stout thorn sacrifices fight behave thicker ensue throat neariah methegammah toah marchedst gier idle memory forerunner recall eunice righteousness jekabzeel assaying herself ruddy suck beneath lasciviousness narrowly trance fuel dominion threatened looked greatness ahuzam reckoned fray eliasaph searchest manifold hasted backs chaldaeans miriam raamiah foam

As anyone can tell from the outputs above, the generated sentences do not make sense and are too long. This might be because add-one smoothing gives significant probability to words that are otherwise extremely unlikely to appear.

Now, I will once again try generating sentences, but this time add-one smoothing will not be used.

In [17]:
for n in range(2, 6):
    print('========== ' + ('bi' if n == 2 else ('tri' if n == 3 else f'{n}-')) + 'gram ==========')
    print()
    
    ngram = [''] * n
    ngram[-2:] = ['i', 'will']
    print(' '.join(ngram[-2:]), end='')

    sentence_len = 2

    start = 0 # for finding the starting point of history
    
    while ngram[-1] not in end_mark_set:
        if 200 < sentence_len:
            print(' <<< The sentence being generated has exceeded 200 tokens. Sentence finishing failed. >>>')
            break
        
        ngram[:-1] = ngram[1:]
        
        if ngram[-2] == '':
            history = '/' * (n - 2)
            history_count = kjv_trie['.'] + kjv_trie['?'] + kjv_trie['!']
        else:
            if 0 < start:
                start -= 1
            
            for i in range(start, n - 1):
                history = '/'.join(ngram[i:-1])
                try:
                    history_count = kjv_trie[history]
                    start = i
                    break
                except KeyError:
                    pass

        p = np.zeros(kjv_v)
        
        for i in range(kjv_v):
            try:
                p[i] = kjv_trie[history + '/' + kjv_wordlist[i]] / history_count
            except KeyError:
                pass
        
        ngram[-1] = kjv_wordlist[random.choice(kjv_v, p=p)]

        if punc_pattern.match(ngram[-1]) or ngram[-2] == '-' or ngram[-2:] == ['’', 's']:
            output = ngram[-1]
        else:
            output = ' ' + ngram[-1]
        print(output, end='')

        sentence_len += 1
    
    print()
    print()

========== bigram ==========

i will also i call unto saul my people, saith the king of judah from chun, to thee the judgment to put away.

========== trigram ==========

i will build again the waters under the sun.

========== 4-gram ==========

i will both lay me down in the pillar of the cloud.

========== 5-gram ==========

i will bring it health and cure, and i will plant them in this land assuredly with my whole heart: be merciful unto me, and keep my sabbaths: i am the lord.



The sentences are much shorter and somewhat recognizable. However, the meaning of the sentences are still either incomprehensible or absurd. This time, new words will be generated deterministically. The most probable word will be generated.

In [18]:
for n in range(2, 6):
    print('========== ' + ('bi' if n == 2 else ('tri' if n == 3 else f'{n}-')) + 'gram ==========')
    print()
    
    ngram = [''] * n
    ngram[-2:] = ['i', 'will']
    print(' '.join(ngram[-2:]), end='')

    sentence_len = 2

    start = 0 # for finding the starting point of history
    
    while ngram[-1] not in end_mark_set:
        if 200 < sentence_len:
            print(' <<< The sentence being generated has exceeded 200 tokens. Sentence finishing failed. >>>')
            break
        
        ngram[:-1] = ngram[1:]
        
        if ngram[-2] == '':
            history = '/' * (n - 2)
            history_count = kjv_trie['.'] + kjv_trie['?'] + kjv_trie['!']
        else:
            if 0 < start:
                start -= 1
            
            for i in range(start, n - 1):
                history = '/'.join(ngram[i:-1])
                try:
                    history_count = kjv_trie[history]
                    start = i
                    break
                except KeyError:
                    pass

        p = np.zeros(kjv_v)
        
        for i in range(kjv_v):
            try:
                p[i] = kjv_trie[history + '/' + kjv_wordlist[i]] / history_count
            except KeyError:
                pass
        
        ngram[-1] = kjv_wordlist[p.argmax()]

        if punc_pattern.match(ngram[-1]) or ngram[-2] == '-' or ngram[-2:] == ['’', 's']:
            output = ngram[-1]
        else:
            output = ' ' + ngram[-1]
        print(output, end='')
        
        sentence_len += 1
    
    print()
    print()

========== bigram ==========

i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i will i <<< The sentence being generated has exceeded 200 tokens. Sentence finishing failed. >>>


========== trigram ==========

i will not be ashamed, and the lord, and the lord, and the lord, and the lord, and the lord, and the lord, and the lord, and the lord, and the lor

This time, most n-gram models ended up generating repeated sequences. It seems that the best method of text generation is random generation without add-one smoothing.

## Universal Declaration of Human Rights

Now, I build n-gram models with the Universal Declaration of Human Rights. This will give us a language model for Modern English.

In [4]:
udhr_eng = UDHREngReader()
udhr_eng_list = []

Before building the models, I parse the text into a list of words and punctuations. This will be convenient for building the models.

In [5]:
while not udhr_eng.is_eof():
    units = udhr_eng.read_sentence().lower().split()
    for u in units:
        while u:
            match = punc_pattern.search(u)
            if match:
                i = match.start()
                if i == 0:
                    punc = u[0]
                    u = u[1:]
                else:
                    first_word = u[:i]
                    punc = u[i]
                    u = u[i+1:]
                    udhr_eng_list.append(first_word)
                udhr_eng_list.append(punc)
            else:
                udhr_eng_list.append(u)
                break

First, I build the unigram model without add-one smoothing.

In [6]:
# Unigram Model

udhr_eng_trie = pygtrie.StringTrie()
udhr_eng_v = 0 # size of the vocabulary
udhr_eng_wordlist = []

for w in udhr_eng_list:
    try:
        udhr_eng_trie[w] += 1
    except KeyError:
        udhr_eng_trie[w] = 1
        udhr_eng_wordlist.append(w)
        udhr_eng_v += 1

print('Size of the vocabulary:', udhr_eng_v)
print('Count of all words:', len(udhr_eng_list))

udhr_eng_unigram_H = 0 # cross entropy

for w in udhr_eng_list:
    p = udhr_eng_trie[w] / len(udhr_eng_list)
    udhr_eng_unigram_H += math.log2(p)
udhr_eng_unigram_H /= -len(udhr_eng_list)

print('Cross entropy:', udhr_eng_unigram_H)
print('Perplexity:', 2 ** udhr_eng_unigram_H)
print()

udhr_eng_unigram_list = sorted(udhr_eng_trie.items(), key=lambda t : t[1], reverse=True)
for pair in udhr_eng_unigram_list:
    p = pair[1] / len(udhr_eng_list)
    print('Word:', pair[0], '| Probability:', p)

Size of the vocabulary: 12557
Count of all words: 917082
Cross entropy: 8.298804717355264
Perplexity: 314.91195501035224

Word: , | Probability: 0.07696040266846367
Word: the | Probability: 0.06970914269389215
Word: and | Probability: 0.05637009558578186
Word: of | Probability: 0.03774798763905518
Word: . | Probability: 0.028567783469744253
Word: to | Probability: 0.014788208687990823
Word: that | Probability: 0.014079438916040223
Word: : | Probability: 0.01384936134391472
Word: in | Probability: 0.013812287232766536
Word: he | Probability: 0.01136103423685123
Word: ; | Probability: 0.011055718027395587
Word: shall | Probability: 0.010727503102230772
Word: unto | Probability: 0.009810464058830072
Word: for | Probability: 0.009782113267952048
Word: i | Probability: 0.00965453470900094
Word: his | Probability: 0.00923908658113451
Word: a | Probability: 0.008916323731138546
Word: lord | Probability: 0.008684065328945504
Word: they | Probability: 0.008042901289088652
Word: be | Probability

# TODO: Finish the UDHR model